In [39]:
!pip install -U google-generativeai
!pip install langchain langchain-community langchain-google-genai sentence-transformers chromadb langchain-text-splitters


In [40]:
import os
import getpass

# Secure input box (hides your API key)
os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔑 Enter your Gemini API key and press Enter Key: ")

# Confirm setup (without showing the key)
if os.environ["GOOGLE_API_KEY"]:
    print("✅ Gemini API Key set successfully (not displayed for security).")
else:
    print("❌ No key was entered. Please try again.")


🔑 Enter your Gemini API key and press Enter Key: ··········
✅ Gemini API Key set successfully (not displayed for security).


In [41]:
import google.generativeai as genai  #testing the model

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-flash")

print(model.generate_content("Hello! Are you working?").text)


Hello! Yes, you could say I am! As an AI, I don't really have "work hours" in the human sense; I'm always "on" and ready to process information and assist users.

So, how can I help you today?


In [42]:
from langchain_community.document_loaders import WebBaseLoader #Load Documents (from the Web)

loader = WebBaseLoader("https://flatsixes.com/porsche-news/porsche-sets-november-19-world-debut-for-cayenne-electric/")
docs = loader.load()
docs


[Document(metadata={'source': 'https://flatsixes.com/porsche-news/porsche-sets-november-19-world-debut-for-cayenne-electric/', 'title': 'Porsche sets November 19 world debut for Cayenne Electric | FLATSIXES', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\nPorsche sets November 19 world debut for Cayenne Electric | FLATSIXES\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFLATSIXESHome\nResources\nAbout\nContact\nSearch\n\nAdvice\n\nConcours & Car Care\nDo It Yourself\nPorsche Maintenance Tips\nPorsche Photography\n\n\nLifeStyle\n\nPorsche Accidents\nHumor\nPorsche Car Shows\nReader’s Ride\nPorsche Clubs\nPorsche Contests\nPorsche Factoids\nPorsche Museum\nPorsche Sport Driving School\nResources\nVideos\n\n\nNews\nCars\n\nFor Sale\nTuned Porsches\nPorsche GT2/GT3\nPorsche 928\nPorsche Prototypes\nPorsche 991\nPorsche Boxster\nPorsche 356\nPorsche 918 

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter #Split Documents into Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=300
)

splits = text_splitter.split_documents(docs)
len(splits)


2

In [44]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(         #Convert Chunks to Embeddings (Local — No API Needed) , no rate limit issue using HuggingFace
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}          #cpu instead of gpu as best for free google colab version and headache free
)

vectordb = Chroma.from_documents(
    splits,
    embedding=embeddings,
    persist_directory="chroma_store"
)

vectordb.persist()                   #“This code stores the chunked documents and their embeddings in a persistent Chroma vector database,
                                      #allowing fast semantic search for the RAG system.”


In [45]:
vectordb._collection.count()


20

In [46]:
retriever = vectordb.as_retriever( #This line converts your Chroma Vector Database into a Retriever object that LangChain can use during RAG.
    search_type="similarity",
    search_kwargs={"k": 3}       #Return the top 3 most relevant chunks for every question.
)


In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI        #This imports the Gemini LLM wrapper for LangChain

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)


In [48]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)
# Create the RetrievalQA RAG pipeline:
# - Uses our Gemini LLM to generate answers
# - Uses the retriever to fetch the top-k most relevant document chunks
# - chain_type="stuff" means all retrieved chunks are combined into a single prompt
# This forms the complete RAG system (Retrieve → Inject Context → Answer)


In [49]:
response = qa_chain.invoke({"query": "When is the Cayenne Electric's world debut?"}) #q1
print(response["result"])


The Cayenne Electric's world debut is on November 19, 2025.


In [50]:
qa_chain.invoke({"query": "Summarize the article in 5 bullet points."})["result"] #q2


'Based on the provided text, here is a summary in 5 bullet points:\n\n*   The site encourages users to subscribe to receive updates, reviews, and giveaways as the "first and only source of original Porsche-related content."\n*   It directs users to check out other Porsche blog posts.\n*   The site acknowledges its "valued sponsors."\n*   It offers a section with product recommendations for Porsche care.\n*   Specific product recommendations include the best car waxes, pressure washers, foam cannons, interior cleaners, and quick detailers.'

In [52]:
response = qa_chain.invoke({"query": "What Model Of Porche are we talking about?"}) #q3
print(response["result"])


We are talking about the **Cayenne Electric** model of Porsche.
